### 1. Dependencies

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import DataParallel

import time
import os
import numpy as np
import json
import cv2
from PIL import Image, ImageOps
import random
from tqdm import tqdm
import operator
import itertools
from scipy.io import  loadmat
import logging
from scipy import signal

from utils import data_transforms
from utils import get_paste_kernel, kernel_map
from utils_logging import setup_logger

### 2. Choose between Recasens or GazeNet

- Idea is you can just swap 
models.recasens, dataloader.recasens, training.train_recasens, etc...
- with the following
models.gazenet, dataloader.gazenet, training.train_gazenet

In [2]:
from models.gazenet import GazeNet
from models.__init__ import save_checkpoint, resume_checkpoint
from dataloader.gazenet import GooDataset, GazeDataset
from training.train_gazenet import train, test, GazeOptimizer

In [3]:
# Logger will save the training and test errors to a .log file 
logger = setup_logger(name='first_logger', 
                      log_dir ='./logs/',
                      log_file='train_gazenet.log',
                      log_format = '%(asctime)s %(levelname)s %(message)s',
                      verbose=True)

### 3. Dataloaders
- Choose between GazeDataset (Gazefollow dataset) or GooDataset (GooSynth/GooReal)
- Set paths to image directories and pickle paths. For Gazefollow, images_dir and test_images_dir should be the same and both lead to the path containing the train and test folders.

In [4]:
# Dataloaders for GazeFollow
batch_size=32
workers=12
testbatchsize=16

images_dir = '/home/eee198/Documents/datasets/GazeFollowData/'
pickle_path = '/home/eee198/Documents/datasets/GazeFollowData/train_annotations.mat'
test_images_dir = '/home/eee198/Documents/datasets/GazeFollowData/'
test_pickle_path = '/home/eee198/Documents/datasets/GazeFollowData/test_annotations.mat'

train_set = GazeDataset(images_dir, pickle_path, 'train')
train_data_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=workers)

val_set = GazeDataset(test_images_dir, test_pickle_path, 'test')
test_data_loader = torch.utils.data.DataLoader(val_set, batch_size=testbatchsize, num_workers=workers, shuffle=False)

In [4]:
# Dataloaders for GOO
batch_size=32
workers=12
testbatchsize=32

images_dir = '/hdd/HENRI/goosynth/1person/GazeDatasets/'
pickle_path = '/hdd/HENRI/goosynth/picklefiles/trainpickle2to19human.pickle'
test_images_dir = '/hdd/HENRI/goosynth/test/'
test_pickle_path = '/hdd/HENRI/goosynth/picklefiles/testpickle120.pickle'

train_set = GooDataset(images_dir, pickle_path, 'train')
train_data_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=workers)

val_set = GooDataset(test_images_dir, test_pickle_path, 'test')
test_data_loader = torch.utils.data.DataLoader(val_set, batch_size=testbatchsize, num_workers=workers, shuffle=False)

172800
19200


### 4. Load Model and Set Training Hyperparameters
- For Gazefollow, the model requires the alexnet_places365 pretrained model, provided here: https://urlzs.com/ytKK3
- When resuming training, set to True and set the resume_path for the saved model.
- Here, logging module is initialized (logger) to save training and testing errors.

In [5]:
# Loads model
net = GazeNet()
net.cuda()

# Hyperparameters
start_epoch = 0
max_epoch = 25
learning_rate = 0.0001

# Initializes Optimizer
gaze_opt = GazeOptimizer(net, learning_rate)
optimizer = gaze_opt.getOptimizer(start_epoch)

# Is training resumed? If so, set the resume_path and set flag to True
# This can also be used to evaluate a model 
resume_training = True
resume_path = './saved_models/gazenet_goo/model_epoch25.pth.tar'
if resume_training :
    net, optimizer, start_epoch = resume_checkpoint(net, optimizer, resume_path)
    test(net, test_data_loader,logger)

  0%|          | 0/600 [00:00<?, ?it/s]

=> loading checkpoint './saved_models/gazenet_goo/model_epoch25.pth.tar'
=> loaded checkpoint './saved_models/gazenet_goo/model_epoch25.pth.tar' (epoch 25)


loss: 0.08515, 0.14883, 0.23398
 17%|█▋        | 102/600 [01:37<04:46,  1.74it/s]loss: 0.06893, 0.11350, 0.18243


 34%|███▍      | 203/600 [02:59<05:29,  1.20it/s]loss: 0.06802, 0.07885, 0.14687


 51%|█████     | 304/600 [04:21<03:56,  1.25it/s]loss: 0.07628, 0.16189, 0.23817


 68%|██████▊   | 405/600 [05:37<00:50,  3.86it/s]loss: 0.06882, 0.09347, 0.16228


 84%|████████▍ | 506/600 [06:58<01:00,  1.55it/s]loss: 0.08128, 0.18208, 0.26337


100%|██████████| 600/600 [08:06<00:00,  1.23it/s]
average loss : [0.06995576 0.12286932 0.19282507]
average error: [ 0.10772795 20.60244929 19.72047437]


In [6]:
pretrained_dict = torch.load('./saved_models/gazenet_goo/model_epoch25.pkl')
model_dict = net.state_dict()
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
model_dict.update(pretrained_dict)
net.load_state_dict(model_dict)

save_path = './saved_models/gazenet_goo/'
save_checkpoint(net, optimizer, 25, save_path)

### 5. Training the Model
- Determine in which epochs do you want to save the model, as you might not want to save every epoch
- Training and test errors can be accessed in the logs directory set up earlier

In [6]:
for epoch in range(start_epoch, max_epoch):
    
    # Update optimizer
    optimizer = gaze_opt.getOptimizer(epoch)

    # Train model
    train(net, train_data_loader, optimizer, epoch, logger)

    # Save model and optimizer
    if epoch > max_epoch-5:
        save_path = './saved_models/temp/'
        save_checkpoint(net, optimizer, epoch+1, save_path)
    
    # Evaluate model
    test(net, test_data_loader, logger)

  3%|▎         | 156/5400 [02:50<1:35:42,  1.10s/it]


KeyboardInterrupt: 